In [2]:
pip install selenium pandas webdriver-manager beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configuración del navegador
options = Options()
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)
driver.get("https://petshelter.miwuki.com/perros-en-adopcion")

# Aceptar cookies
try:
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "fc-cta-consent"))).click()
    print("✅ Cookies aceptadas")
except:
    print("⚠️ No se encontró el botón de cookies")

# Cargar más perros haciendo clic en "Continuar viendo"
for i in range(10):
    try:
        btn = wait.until(EC.element_to_be_clickable((By.ID, "loadBtn")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
        time.sleep(2)
        btn.click()
        print(f"🔁 Clic en 'Continuar viendo' #{i+1}")
        time.sleep(6)
    except:
        print("⛔ Botón 'Continuar viendo' no disponible o ya no hay más perros")
        break

# Obtener las imágenes clicables
imagenes = driver.find_elements(By.XPATH, "//img[starts-with(@alt, 'Adopta a')]")
print(f"🐶 Se encontraron {len(imagenes)} perfiles")

dogs_data = []
main_window = driver.current_window_handle

for i, img in enumerate(imagenes):
    try:
        driver.switch_to.window(main_window)
        time.sleep(1)

        imagenes_actualizadas = driver.find_elements(By.XPATH, "//img[starts-with(@alt, 'Adopta a')]")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", imagenes_actualizadas[i])
        time.sleep(1)
        imagenes_actualizadas[i].click()
        time.sleep(3)

        new_tab = [w for w in driver.window_handles if w != main_window][0]
        driver.switch_to.window(new_tab)
        profile_url = driver.current_url
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Nombre limpio (solo texto directo del h1 sin incluir íconos de sexo)
        nombre_tag = soup.select_one("div.info-mascota h1")
        nombre = ""
        if nombre_tag:
            for elem in nombre_tag.contents:
                if elem.name in ["span", "i"]:
                    continue
                if isinstance(elem, str):
                    nombre += elem.strip()
        nombre = nombre.strip()

        # Sexo desde el icono (title)
        sexo_tag = soup.select_one("div.info-mascota i[title]")
        sexo = sexo_tag["title"] if sexo_tag else ""

        # Protectora
        protectora_tag = soup.select_one("div.owner.mb-3 h1")
        protectora = protectora_tag.get_text(strip=True) if protectora_tag else ""

        # Ubicación
        ubicacion_tag = soup.select_one("div.info-item")
        ubicacion = ubicacion_tag.get_text(strip=True) if ubicacion_tag else ""

        # Datos: especie, edad, peso, tamaño, nivel actividad
        datos = {}
        for div in soup.select("div.row.detalles div.col-6.col-md-3"):
            span = div.find("span")
            if span:
                key = span.get_text(strip=True)
                span.extract()
                value = div.get_text(strip=True)
                datos[key] = value.strip()

        especie = datos.get("Especie", "")
        edad = datos.get("Edad", "")
        peso = datos.get("Peso", "")
        tamano = datos.get("Tamaño", "")
        actividad = datos.get("Nivel Actividad", "")

        # Edad real (años + meses combinados)
        edad_real = ""
        for div in soup.select("div.row.detalles div.col-6.col-md-3"):
            if div.find("small", string=" años") or div.find("small", string=" meses"):
                texto = div.get_text(strip=True)
                edad_real = texto.replace("años", "años ").replace("meses", "meses").strip()
                break

        # ¿Cómo soy? (cualidades)
        cualidades_tags = soup.select("div.pills span")
        cualidades = [tag.get_text(strip=True) for tag in cualidades_tags if tag.get_text(strip=True)]
        cualidades_str = ", ".join(cualidades)

        # Mi historia
        historia_tag = soup.select_one("h2:contains('Mi historia') + p")
        historia = historia_tag.get_text(strip=True) if historia_tag else ""

        dogs_data.append({
            "Nombre": nombre,
            "Sexo": sexo,
            "Protectora": protectora,
            "Ubicación": ubicacion,
            "Especie": especie,
            "Edad": edad,
            "Edad Real": edad_real,
            "Peso": peso,
            "Tamaño": tamano,
            "Nivel Actividad": actividad,
            "¿Cómo soy?": cualidades_str,
            "Mi historia": historia,
            "URL": profile_url
        })

        print(f"✔️ {nombre} extraído correctamente")
        driver.close()
        driver.switch_to.window(main_window)

    except Exception as e:
        print(f"❌ Error con perfil {i+1}: {e}")
        try:
            driver.close()
            driver.switch_to.window(main_window)
        except:
            pass
        continue

driver.quit()

# Guardar CSV
df = pd.DataFrame(dogs_data)
df.to_csv("perros_perfil_miwuki.csv", index=False)
print("📁 Datos guardados en 'perros_perfil_miwuki.csv'")

✅ Cookies aceptadas
🔁 Clic en 'Continuar viendo' #1
🔁 Clic en 'Continuar viendo' #2
🔁 Clic en 'Continuar viendo' #3
🔁 Clic en 'Continuar viendo' #4
🔁 Clic en 'Continuar viendo' #5
🔁 Clic en 'Continuar viendo' #6
🔁 Clic en 'Continuar viendo' #7
🔁 Clic en 'Continuar viendo' #8
🔁 Clic en 'Continuar viendo' #9
🔁 Clic en 'Continuar viendo' #10
🐶 Se encontraron 396 perfiles


/opt/anaconda3/lib/python3.12/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


✔️ Kally extraído correctamente
✔️ Adam extraído correctamente
✔️ Nala extraído correctamente
✔️ Mireia extraído correctamente
✔️ Melvin extraído correctamente
✔️ Wangwang extraído correctamente
✔️ Rabote extraído correctamente
✔️ Sammy extraído correctamente
✔️ Rita extraído correctamente
✔️ Noa extraído correctamente
✔️ Harpo extraído correctamente
✔️ Dolça extraído correctamente
✔️ Zoe extraído correctamente
✔️ Charlotte extraído correctamente
✔️ Blanca extraído correctamente
✔️ Lotta extraído correctamente
✔️ Roco extraído correctamente
✔️ Enric extraído correctamente
✔️ Emilio extraído correctamente
✔️ Melocotón extraído correctamente
✔️ Blanco extraído correctamente
✔️ Troncho extraído correctamente
✔️ Belma extraído correctamente
✔️ Black extraído correctamente
✔️ Looby extraído correctamente
✔️ Widow extraído correctamente
✔️ Mili extraído correctamente
✔️ Yako extraído correctamente
✔️ Timón extraído correctamente
✔️ Bico extraído correctamente
✔️ Toby y Cloe extraído correcta

In [63]:
import pandas as pd
df = pd.read_csv("perros_perfil_miwuki.csv")
df

,Nombre,Sexo,Protectora,Ubicación,Especie,Edad,Edad Real,Peso,Tamaño,Nivel Actividad,¿Cómo soy?,Mi historia,URL
0,Kally,Hembra,Red de Protección Animal Perros del AlmaProtec...,"Buenos Aires, Argentina",Perro,Adulto,NaN,0 kg,Mediano,Sin determinar,"Bueno con otros perros, Cariñoso, Tímido",Es tímida y dulce. Convive con otros perros si...,https://petshelter.miwuki.com/c/QWJK8Amvt3
1,Adam,Macho,Vidas PeludasProtectora,"Granada, España",Perro,Cachorro,NaN,kg,Grande,Alta,"Bueno en casa, Bueno en el coche, Bueno con ga...",Adam fue cedido por un galguero….. Es un perri...,https://petshelter.miwuki.com/c/RwRCnxwkea
2,Nala,Hembra,Animales Abandonados GranadaProtectora,"Granada, España",Perro,Cachorro,NaN,7 kg,Pequeño,Media,"Bueno con niños, Bueno con otros animales, Bue...","NALA FUE DEJADA EN LA PUERTA DE EL REFUGIO, ES...",https://petshelter.miwuki.com/c/yjNh9eIwqa
3,Mireia,Hembra,Spax Protectora De Animales De XàtivaProtectora,"Valencia, España",Perro,Adulto,NaN,20 kg,Mediano,Media,"Bueno en casa, Bueno con otros animales, Bueno...",Esta perrita entró en el refugio junto a su he...,https://petshelter.miwuki.com/c/2sL9QlXPA3
4,Melvin,Macho,ABYDA JaénProtectora,"Jaén, España",Perro,Joven,NaN,0 kg,Grande,Media,"Bueno con niños, Bueno con otros perros, Bueno...",Melvin es un perrazo con alma de cachorro. Pue...,https://petshelter.miwuki.com/c/uD7cBRHAox
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,Emilio,Macho,CANTILEROSProtectora,"Madrid, España",Perro,Joven,NaN,0 kg,Desconocido,Sin determinar,"Buen temperamento, Me gusta la compañía",NaN,https://petshelter.miwuki.com/c/9CBBNDZ6od
391,Roki,Macho,Gema del Carmen A.Particular,"Ciudad de México, México",Perro,Joven,NaN,4 kg,Pequeño,Media,"Bueno con otros perros, Bueno con otras person...","soy un perro jugueton y gruñón a la vez,pero n...",https://petshelter.miwuki.com/c/Wpi0BOAEEX
392,Igor,Macho,El Hogar De MikaProtectora,"Barcelona, España",Perro,Cachorro,NaN,kg,Mediano,Sin determinar,"Bueno en casa, Bueno en el coche, Bueno con ni...",Recogido de una zona rural. Es un cachorron aú...,https://petshelter.miwuki.com/c/mZclG3KQP3
393,Harry,Macho,Alma Delia R.Particular,"Ciudad de México, México",Perro,Joven,NaN,267 kg,Mediano,Alta,"Bueno en casa, Bueno en el coche, Bueno con ga...","Golden Retriever, color dorado",https://petshelter.miwuki.com/c/X0vfaVb2Sz


In [64]:
df.head()

,Nombre,Sexo,Protectora,Ubicación,Especie,Edad,Edad Real,Peso,Tamaño,Nivel Actividad,¿Cómo soy?,Mi historia,URL
0,Kally,Hembra,Red de Protección Animal Perros del AlmaProtec...,"Buenos Aires, Argentina",Perro,Adulto,NaN,0 kg,Mediano,Sin determinar,"Bueno con otros perros, Cariñoso, Tímido",Es tímida y dulce. Convive con otros perros si...,https://petshelter.miwuki.com/c/QWJK8Amvt3
1,Adam,Macho,Vidas PeludasProtectora,"Granada, España",Perro,Cachorro,NaN,kg,Grande,Alta,"Bueno en casa, Bueno en el coche, Bueno con ga...",Adam fue cedido por un galguero….. Es un perri...,https://petshelter.miwuki.com/c/RwRCnxwkea
2,Nala,Hembra,Animales Abandonados GranadaProtectora,"Granada, España",Perro,Cachorro,NaN,7 kg,Pequeño,Media,"Bueno con niños, Bueno con otros animales, Bue...","NALA FUE DEJADA EN LA PUERTA DE EL REFUGIO, ES...",https://petshelter.miwuki.com/c/yjNh9eIwqa
3,Mireia,Hembra,Spax Protectora De Animales De XàtivaProtectora,"Valencia, España",Perro,Adulto,NaN,20 kg,Mediano,Media,"Bueno en casa, Bueno con otros animales, Bueno...",Esta perrita entró en el refugio junto a su he...,https://petshelter.miwuki.com/c/2sL9QlXPA3
4,Melvin,Macho,ABYDA JaénProtectora,"Jaén, España",Perro,Joven,NaN,0 kg,Grande,Media,"Bueno con niños, Bueno con otros perros, Bueno...",Melvin es un perrazo con alma de cachorro. Pue...,https://petshelter.miwuki.com/c/uD7cBRHAox


In [65]:
df.shape

(395, 13)